In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from pydrake.math import RigidTransform, RollPitchYaw
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_ur5e_and_table_to_builder
from cloth_tools.drake.visualization import add_meshcat_triad

In [ ]:
tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_index, gripper_index = add_ur5e_and_table_to_builder(robot_diagram_builder)
diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

In [ ]:
transform = RigidTransform(p=[0.25, 0, 0.4], rpy=RollPitchYaw([np.pi, 0, 0]))
tcp_pose_0 = np.ascontiguousarray(transform.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame", X_W_Triad=transform)

with np.printoptions(precision=10, suppress=True):
    print("tcp_pose_0")
    print(tcp_pose_0)

In [ ]:
from ur_analytic_ik import ur5e

from cloth_tools.drake.visualization import publish_ik_solutions

# Important note:
# Drake TCP poses can be passed directly to the IK only when the robot base (as defined by the UR control box)
# is aligned with the world frame.
solutions = ur5e.inverse_kinematics_with_tcp(tcp_pose_0, tcp_transform)
publish_ik_solutions(solutions, 2.0, meshcat, diagram, context, arm_index)

In [ ]:
tcp_pose_1 = np.identity(4)
X = np.array([-1.0, 0.0, 0.0])
Y = np.array([0.0, 1.0, 0.0])
Z = np.array([0.0, 0.0, -1.0])
top_down_orientation = np.column_stack([X, Y, Z])
translation = np.array([-0.2, -0.2, 0.2])

tcp_pose_1[:3, :3] = top_down_orientation
tcp_pose_1[:3, 3] = translation

with np.printoptions(precision=3, suppress=True):
    print("tcp_pose_1")
    print(tcp_pose_1)

In [ ]:
add_meshcat_triad(meshcat, "tcp_pose_1", X_W_Triad=RigidTransform(tcp_pose_1))

In [ ]:
with np.printoptions(precision=3, suppress=True):
    print("inverse_kinematics")
    print(ur5e.inverse_kinematics(tcp_pose_1))

In [ ]:
solutions_2 = ur5e.inverse_kinematics_with_tcp(tcp_pose_1, tcp_transform)
publish_ik_solutions(solutions_2, 2.0, meshcat, diagram, context, arm_index)